In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = sns.load_dataset('penguins')
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [3]:
df.shape

(344, 7)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [5]:
df.isnull().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
df.isnull().sum()

species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64

*Feature Engineering*


In [8]:
df.sex.unique()


array(['Male', 'Female'], dtype=object)

In [9]:
pd.get_dummies(df['sex']).head()

,Female,Male
0,False,True
1,True,False
2,True,False
4,True,False
5,False,True


In [10]:
sex = pd.get_dummies(df['sex'],drop_first=True)
sex.head()


,Male
0,True
1,False
2,False
4,False
5,True


# Applying one-hot encoding to island

In [11]:
df.island.unique()

array(['Torgersen', 'Biscoe', 'Dream'], dtype=object)

In [12]:
pd.get_dummies(df['island']).head()


,Biscoe,Dream,Torgersen
0,False,False,True
1,False,False,True
2,False,False,True
4,False,False,True
5,False,False,True


In [13]:
island = pd.get_dummies(df['island'], drop_first=True)
island.head(5)

,Dream,Torgersen
0,False,True
1,False,True
2,False,True
4,False,True
5,False,True



Concatenate the above two data frames to the original df

In [14]:
new_data = pd.concat([df,island, sex], axis=1)

In [15]:
new_data.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,Dream,Torgersen,Male
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male,False,True,True
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female,False,True,False
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female,False,True,False
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female,False,True,False
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male,False,True,True


In [16]:
new_data.drop(['sex','island'], axis=1, inplace=True)

In [17]:
new_data.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,Dream,Torgersen,Male
0,Adelie,39.1,18.7,181.0,3750.0,False,True,True
1,Adelie,39.5,17.4,186.0,3800.0,False,True,False
2,Adelie,40.3,18.0,195.0,3250.0,False,True,False
4,Adelie,36.7,19.3,193.0,3450.0,False,True,False
5,Adelie,39.3,20.6,190.0,3650.0,False,True,True


In [18]:
Y = new_data.species
Y.head()

0    Adelie
1    Adelie
2    Adelie
4    Adelie
5    Adelie
Name: species, dtype: object

In [19]:
Y.unique()

array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object)

In [20]:
Y=Y.map({'Adelie' : 0,'Chinstrap' : 1, 'Gentoo':2})
Y.head()

0    0
1    0
2    0
4    0
5    0
Name: species, dtype: int64

In [21]:
new_data.drop('species', inplace=True, axis=1)
new_data.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,Dream,Torgersen,Male
0,39.1,18.7,181.0,3750.0,False,True,True
1,39.5,17.4,186.0,3800.0,False,True,False
2,40.3,18.0,195.0,3250.0,False,True,False
4,36.7,19.3,193.0,3450.0,False,True,False
5,39.3,20.6,190.0,3650.0,False,True,True


In [22]:
X=new_data

Splitting the data set into Training & Test Data

# Spilliting the data set into Training and Test Data

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3, random_state=0)

In [24]:
print('X_train',X_train.shape)
print('X_test',X_test.shape)
print('Y_train',y_train.shape)
print('Y_test',y_test.shape)


X_train (233, 7)
X_test (100, 7)
Y_train (233,)
Y_test (100,)


# Training Random Forest Classification on Training Set

In [25]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 5, criterion = 'entropy', random_state = 0)
classifier. fit(X_train, y_train)


RandomForestClassifier(criterion='entropy', n_estimators=5, random_state=0)

# Predicting the Test Results

In [26]:
y_pred = classifier.predict(X_test)
y_pred

array([0, 0, 2, 0, 0, 0, 1, 2, 2, 1, 2, 0, 0, 1, 0, 0, 2, 0, 1, 0, 0, 0,
       2, 2, 2, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 1, 0, 2, 2, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 1, 0, 2, 0, 0,
       2, 2, 1, 2, 2, 1, 2, 1, 0, 2, 0, 2, 0, 2, 1, 2, 2, 2, 1, 2, 1, 0,
       0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 2, 2], dtype=int64)

# Confusion Matrix

In [27]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report,accuracy_score

In [28]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[48  0  0]
 [ 2 14  0]
 [ 0  0 36]]


In [29]:
accuracy_score(y_test, y_pred)

0.98

In [30]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        48
           1       1.00      0.88      0.93        16
           2       1.00      1.00      1.00        36

    accuracy                           0.98       100
   macro avg       0.99      0.96      0.97       100
weighted avg       0.98      0.98      0.98       100



# Try with Different number of trees and gini criteria


In [31]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 7, criterion = 'gini', random_state = 0)
classifier. fit(X_train, y_train)

RandomForestClassifier(n_estimators=7, random_state=0)

In [33]:
y_pred = classifier.predict(X_test)

In [34]:
accuracy_score(y_test, y_pred)

0.99

# with more trees the model gives 99% accuracy